In [28]:
import numpy as np
import pandas as pd
from itertools import chain, combinations
import json
from tqdm import tqdm
import math

In [69]:
# gets the graphs created inside "creatingGraphs"
df = pd.read_csv('linkedGraphs.csv', engine='python', header=None) 

In [70]:
# deleting unrelevant stuff and renaming
df = df.drop([0], axis = 0)
df = df.rename(columns={0: "Matrix"})

In [71]:
df

,Matrix
1,"[[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]..."
2,"[[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]..."
3,"[[1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0]..."
4,"[[1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1]..."
5,"[[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]..."
...,...
996,"[[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1]..."
997,"[[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]..."
998,"[[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0]..."
999,"[[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]..."


In [80]:
size_of_matrixs = len(json.loads(df.iloc[0]["Matrix"]))
amount_of_matrixs = len(df)

In [81]:
pset_dict = dict()
'''
Returns the powerset of the list given
Since this function can take time, we keep a cache, "pset_dict"
that will keep the powersets we already calculated.
I use the len of the graph as a key, because this powerset
function is only used in sets that starts with 0 and increase their
elements by 1.
'''
def powerset(s):
    x = len(s)
    if x in pset_dict:
        return pset_dict[x]
    pset = []
    for i in range(1 << x):
        pset.append([s[j] for j in range(x) if (i & (1 << j))])
    pset_dict[x] = pset
    return pset

In [87]:
'''
Gets the minimum vertex cover of a given graph.
'''
def get_min_vertex_cover(graph):
    vert = [i for i in range(size_of_matrixs)]
    subsets = list(powerset(vert))
    subsets = sorted(subsets, key=len) 
    current_subset = []
    for subset in subsets:
        # choose the version needed
        if is_vertex_cover2(graph, subset):
            return subset  

In [88]:
'''
Checks if the subset given is a vertex cover for the graph given.
'''
def is_vertex_cover(graph, subset):
    graph_visited = len(graph) * [False]
    # for each vertex in subset, mark as true
    for i in subset: 
        graph_visited[i] = True
    # for each edge, checks that one of the vertex(at least) is True.
    for u in range(int(len(graph) / size_of_matrixs)):
        for v in range(u):
            if graph[u*size_of_matrixs + v] == 1 and not graph_visited[u] and not graph_visited[v]:
                return False
    return True

In [89]:
'''
Checks if the subset given is a vertex cover for the graph given.

This version is used when we use a new df that we pulled from "creatingGraphs"
if we want to create new solutions to a df where the graphs are already represented
as one array, we use the other version
'''
def is_vertex_cover2(graph, subset):
    graph_visited = len(graph) * [False]
    # for each vertex in subset, mark as true
    for i in subset: 
        graph_visited[i] = True
    # for each edge, checks that one of the vertex(at least) is True.
    for u in range(len(graph)):
        for v in range(u):
            if graph[u][v] == 1 and not graph_visited[u] and not graph_visited[v]:
                return False
    return True

In [90]:
sols = []
# solves the problem
for ix in tqdm(df.index):
    if ix == 100:
        break;
    current = df.iloc[ix-1]
    sol = get_min_vertex_cover(json.loads(current['Matrix']))
    sols.append(sol)

 10%|███████▉                                                                        | 99/1000 [00:04<00:44, 20.16it/s]


In [57]:
# adds the solution to the df
df['Solution'] = sols

In [92]:
df

,Matrix
1,"[[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]..."
2,"[[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]..."
3,"[[1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0]..."
4,"[[1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1]..."
5,"[[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]..."
...,...
996,"[[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1]..."
997,"[[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]..."
998,"[[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0]..."
999,"[[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]..."


In [ ]:
'''
Gets an array with numbers(vertex) and returns a new array as
hot one encoding in the size of the matrix.
'''
def hot_one_encoding(array):
    new_array = [0 for i in range(size_of_matrixs)]
    for i in array:
        new_array[i] = 1
    return new_array

In [ ]:
new_sols = [0 for i in range(amount_of_matrixs)]
# turns the solutions to hot one encoding 
for i in range(len(new_sols)):
    new_sols[i] = hot_one_encoding(sols[i])
new_sols

In [ ]:
df['Solution'] = new_sols

In [ ]:
df

In [ ]:
new_matrix = [0 for i in range(amount_of_matrixs)]
# transform the matrix to 1d list instead of 2d list
for i in range(amount_of_matrixs):
    current = df.iloc[i]
    flat_list = [item for sublist in json.loads(current['Matrix']) for item in sublist]
    new_matrix[i] = flat_list

In [ ]:
df["Matrix"] =  new_matrix

In [ ]:
amount_of_sol = [0 for i in range(amount_of_matrixs)]
# counts the amount of vertex in the solution found.
for i in range(amount_of_matrixs):
    current = df.iloc[i]
    amount = sum(current['Solution'])
    amount_of_sol[i] = amount

In [ ]:
df["Amount"] =  amount_of_sol

In [ ]:
# save the new df in "linkedGraphsWithSolutions"
df.to_csv('linkedGraphsWithSolutions.csv', index=False)